In [1]:
import json
import joblib
from pathlib import Path

from econml.dml import LinearDML, DML
from econml.inference import BootstrapInference
from lightgbm import LGBMRegressor
import numpy as np
import pandas as pd
import tqdm

from sklearn.linear_model import RidgeCV
from sklearn.model_selection import GroupKFold

import seaborn as sns

In [2]:
models = Path("models") / "foreign_ownership"
models.mkdir(parents=True, exist_ok=True)

data_path = Path("../data")
file_name = "ToAnalysis_Winsorized_2015_2023_With_Profit_Asset.csv"
file_path = data_path / file_name

df = pd.read_csv(file_path, parse_dates=['start date'])
df.columns = [col.replace(" ", "_").lower() for col in df.columns]
df = df.sort_values(by=['ticker', 'start_date'])

df["log_tobin_q_winsor"] = np.log(df['tobin_q_winsor'])
df["year"] = df.start_date.dt.year
df["roa"] = df["net_income"].div(df["total_assets"])

In [3]:
df = df.assign(lag_log_tobin_q_winsor=df.groupby(by=['ticker']).log_tobin_q_winsor.shift(-1))

In [4]:
year = pd.get_dummies(df['year'], dtype='int')
year = year.drop(2015, axis=1)
industry = pd.get_dummies(df['industry_name'], dtype='int')
industry = industry.drop("その他製品", axis=1)

In [5]:
Y_cols = ['lag_log_tobin_q_winsor']
W_cols = [
    'board_size', 
    'log_firm_age', 
    'log_sales',
    'sales_growth', 
    'foreign_ownership', 
    'managerial_ownership',
    'tangible_assets', 
    'leverage',
    'roa'
]
X_cols = []
T_cols = ['female_director_ratio']
# T_cols = ['female_director']
G_cols = ['ticker']

In [10]:
mundlak_W = df.groupby(by='ticker')[W_cols].transform("mean")
mundlak_W.columns = [f"{col}_mean" for col in mundlak_W.columns]

mundlak_T = df.groupby(by='ticker')[T_cols].transform("mean")
mundlak_T.columns = [f"{col}_mean" for col in mundlak_T.columns]

In [11]:
# コントロール変数
W = df[W_cols]
# W = (W - W.mean()) / W.std()
W = W.join(mundlak_T).join(mundlak_W)
W = W.join(year).join(industry)

# 説明変数
X = df[X_cols]

# 出力
Y = df[Y_cols]

# 介入
T = df[T_cols]

# Groups
G = df['ticker']

tmp = pd.concat((W, X, Y, T, G), axis=1).dropna(how='any', axis=0)

W = W.loc[tmp.index]
X = X.loc[tmp.index]
Y = Y.loc[tmp.index]
T = T.loc[tmp.index]
G = G.loc[tmp.index]

In [12]:
cv = GroupKFold(n_splits=5) 

In [13]:
import optuna
# from optuna_integration.sklearn import SklearnPruningCallback
# from optuna.integration.sklearn import SklearnPruningCallback
from optuna.samplers import TPESampler

from lightgbm import LGBMRegressor
from sklearn.metrics import mean_squared_error
import numpy as np
import pandas as pd

# --- 評価指標 ---
def rmse(y_true, y_pred):
    return mean_squared_error(y_true, y_pred, squared=False)

# --- Optuna の目的関数 ---
def objective(trial):
    # ハイパーパラメータ空間の定義
    params = {
        "learning_rate": trial.suggest_float("learning_rate", 1e-3, 0.3, log=True),
        "n_estimators": trial.suggest_int("n_estimators", 200, 2000, step=200),
        "num_leaves": trial.suggest_int("num_leaves", 8, 1024, log=True),
        "max_depth": trial.suggest_int("max_depth", 3, 16),
        "min_child_samples": trial.suggest_int("min_child_samples", 5, 100),
        "subsample": trial.suggest_float("subsample", 0.5, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.5, 1.0),
        "reg_alpha": trial.suggest_float("reg_alpha", 1e-8, 10.0, log=True),
        "reg_lambda": trial.suggest_float("reg_lambda", 1e-8, 10.0, log=True),
        "random_state": 42,
        "force_row_wise": True,
        "verbose": -1,
    }

    model = LGBMRegressor(**params)

    # グループ付きクロスバリデーション
    fold_rmse = []
    for train_idx, test_idx in cv.split(Y, W, G):
        X_train = W.iloc[train_idx]
        y_train = Y.iloc[train_idx]
        X_valid = W.iloc[test_idx]
        y_valid = Y.iloc[test_idx]

        # early stopping＋Optuna プルーナー
#        pruning_cb = SklearnPruningCallback(trial, "l2")  # LightGBM は "l2"=RMSE
        model.fit(
            X_train.values,
            y_train.values,
            eval_set=[(X_valid.values, y_valid.values)],
            eval_metric="rmse",
#            callbacks=[pruning_cb],
#            early_stopping_rounds=50,
        )

        y_pred = model.predict(X_valid.values)
        fold_rmse.append(rmse(y_valid.values, y_pred))

    # 目的関数値（平均 RMSE）
    return np.mean(fold_rmse)

# --- Optuna Study の作成と実行 ---
study = optuna.create_study(
    direction="minimize",
    sampler=TPESampler(seed=42),  # 再現性確保
)
study.optimize(objective, n_trials=100, timeout=None, show_progress_bar=True)

print("Best RMSE:", study.best_value)
print("Best params:")
for k, v in study.best_params.items():
    print(f"  {k}: {v}")

# --- ベストパラメータで全データ学習 ---
best_model = LGBMRegressor(**study.best_params, force_row_wise=True, verbose=-1)
best_model.fit(W.values, Y.values)

[I 2025-07-15 18:27:24,269] A new study created in memory with name: no-name-0190b85a-95c1-489d-90d4-99f21e228955


  0%|          | 0/100 [00:00<?, ?it/s]

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
Converting column-vector to 1d array
'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
Converting column-vector to 1d array
'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
Converting column-vector to 1d array
'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
A column-vector y was passed when 

[I 2025-07-15 18:29:30,245] Trial 0 finished with value: 0.2882196102319833 and parameters: {'learning_rate': 0.008468008575248327, 'n_estimators': 2000, 'num_leaves': 274, 'max_depth': 11, 'min_child_samples': 19, 'subsample': 0.5779972601681014, 'colsample_bytree': 0.5290418060840998, 'reg_alpha': 0.6245760287469893, 'reg_lambda': 0.002570603566117598}. Best is trial 0 with value: 0.2882196102319833.


'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
Converting column-vector to 1d array
'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
Converting column-vector to 1d array
'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
Converting column-vector to 1d array
'squared' is deprecated in version

...

'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
Converting column-vector to 1d array
'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
Converting column-vector to 1d array
'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
Converting column-vector to 1d array
'squared' is deprecated in version

[I 2025-07-15 19:29:09,985] Trial 98 finished with value: 0.28610500293216895 and parameters: {'learning_rate': 0.019465919235072838, 'n_estimators': 2000, 'num_leaves': 47, 'max_depth': 9, 'min_child_samples': 26, 'subsample': 0.7228228197989188, 'colsample_bytree': 0.5727453038563827, 'reg_alpha': 8.479431094516582e-05, 'reg_lambda': 3.585787229923926e-06}. Best is trial 33 with value: 0.2811798758048984.


'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
Converting column-vector to 1d array
'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
Converting column-vector to 1d array
'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
Converting column-vector to 1d array
'squared' is deprecated in version

[I 2025-07-15 19:29:40,479] Trial 99 finished with value: 0.2817889561127439 and parameters: {'learning_rate': 0.010159682460440446, 'n_estimators': 2000, 'num_leaves': 22, 'max_depth': 8, 'min_child_samples': 15, 'subsample': 0.7091012086699239, 'colsample_bytree': 0.5399076394387622, 'reg_alpha': 6.069890222966946e-06, 'reg_lambda': 2.8402000991463783e-07}. Best is trial 33 with value: 0.2811798758048984.
Best RMSE: 0.2811798758048984
Best params:
  learning_rate: 0.008322008936037604
  n_estimators: 1600
  num_leaves: 35
  max_depth: 12
  min_child_samples: 17
  subsample: 0.8602174461805033
  colsample_bytree: 0.5012509783557219
  reg_alpha: 3.914513681756259e-07
  reg_lambda: 0.0013367096237559028


LGBMRegressor(colsample_bytree=0.5012509783557219, force_row_wise=True,
              learning_rate=0.008322008936037604, max_depth=12,
              min_child_samples=17, n_estimators=1600, num_leaves=35,
              reg_alpha=3.914513681756259e-07, reg_lambda=0.0013367096237559028,
              subsample=0.8602174461805033, verbose=-1)